# Analysis-1:

Identifying the best airlines based on : Highest Ratio of (Operated flights)/(Scheduled flights)
                                         Flight speed
                                         Average arrival delay
                                         Flight volume
                                         Taxi In and Out Time

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [16]:
# Reading data from 'Data' folder

cwd_path = os.getcwd() 
root_dir_path=os.path.abspath(os.path.join(cwd_path, os.pardir)) #To get the parent directory of cwd
root=root_dir_path.replace("\\", "/")
if not os.path.exists(root+"/"+'Data'): #To check for Data folder
    print('Path does not exist.')
else:
    if not os.path.exists(root+"/"+'Data'+"/"+'Flights'): #To check for Flights folder
        print('Path does not exist.')
    else:
        Path=root+"/"+'Data'+"/"+'Flights'
        if os.path.exists(Path+"/"+'flights.csv'): #To check for flights.csv
            Flights = pd.read_csv(Path+"/"+'flights.csv',sep=",")
        if os.path.exists(Path+"/"+'airlines.csv'): #To check for airlines.csv
            Airlines = pd.read_csv(Path+"/"+'airlines.csv',sep=",")
        if os.path.exists(Path+"/"+'airports.csv'): #To check for airports.csv
            Airports = pd.read_csv(Path+"/"+'airports.csv',sep=",")

E:\Installed_softwares\AnacondaPython\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
Flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [387]:
Airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [388]:
Airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [18]:
#Adding airline name to datafame
Airlines_dict = dict(zip(Airlines['IATA_CODE'],Airlines['AIRLINE']))
Flights['AIRLINE_NAME'] = Flights['AIRLINE'].apply(lambda x: Airlines_dict[x])
Flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE_NAME
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Alaska Airlines Inc.
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US Airways Inc.
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Alaska Airlines Inc.


In [50]:
#Operated/Scheduled flights in percentage for each airline

Flights.CANCELLED.unique() #0 for not cancelled ,1 for cancelled
#Each flight is either cancelled or operated.

Airlines_Ranking = pd.DataFrame(Flights.groupby('AIRLINE_NAME').count()['SCHEDULED_DEPARTURE'])
Airlines_Ranking['CANCELLED'] = pd.DataFrame(Flights.groupby('AIRLINE_NAME').sum()['CANCELLED'])
Airlines_Ranking['OPERATED']=Airlines_Ranking['SCHEDULED_DEPARTURE']-Airlines_Ranking['CANCELLED']
Airlines_Ranking['OPERATED_SCHEDULED_%']=Airlines_Ranking['OPERATED']/Airlines_Ranking['SCHEDULED_DEPARTURE']*100
Airlines_Ranking.drop(Airlines_Ranking.columns[[0,1,2]],axis=1,inplace=True)
Airlines_Ranking=Airlines_Ranking.reset_index()
Airlines_Ranking.head()


,AIRLINE_NAME,OPERATED_SCHEDULED_%
0,Alaska Airlines Inc.,99.612221
1,American Airlines Inc.,98.495972
2,American Eagle Airlines Inc.,94.900418
3,Atlantic Southeast Airlines,97.337131
4,Delta Air Lines Inc.,99.563411


In [51]:
#Average flight speed of each airline

#Speed = Distance/ Time taken

# Flights['DISTANCE'] is given in miles
# Flights['AIR_TIME'] is given in minutes

#We are calulating the speed in miles/hour

Flights_with_Distance_Airtime=Flights.dropna(subset=['DISTANCE','AIR_TIME']) #Dropping rows with null values
#Calculating the speed in miles/hour
Flights_with_Distance_Airtime['FLIGHT_SPEED'] = 60*Flights_with_Distance_Airtime['DISTANCE']/Flights_with_Distance_Airtime['AIR_TIME']

#Dataframe with average flight speed for each airlines
df =pd.DataFrame(Flights_with_Distance_Airtime.groupby('AIRLINE_NAME').mean()['FLIGHT_SPEED'])
df=df.reset_index()
df = df.rename(columns={'FLIGHT_SPEED': 'AVERAGE_FLIGHT_SPEED(miles/hour)'})

#Merging df to Airlines_Ranking
Airlines_Ranking=pd.merge(Airlines_Ranking,df,how='inner')
Airlines_Ranking.head()

E:\Installed_softwares\AnacondaPython\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,AIRLINE_NAME,OPERATED_SCHEDULED_%,AVERAGE_FLIGHT_SPEED(miles/hour)
0,Alaska Airlines Inc.,99.612221,435.510696
1,American Airlines Inc.,98.495972,427.679334
2,American Eagle Airlines Inc.,94.900418,360.899443
3,Atlantic Southeast Airlines,97.337131,364.683738
4,Delta Air Lines Inc.,99.563411,419.433004


In [52]:
#Average Arrival Delay of each airline in minutes

Flights_with_Arrival_Delay=Flights.dropna(subset=['ARRIVAL_DELAY']) #Dropping rows with null values

#Dataframe with average arrival delay for each airlines
df_arrival_delay =pd.DataFrame(Flights_with_Arrival_Delay.groupby('AIRLINE_NAME').mean()['ARRIVAL_DELAY'])
df_arrival_delay=df_arrival_delay.reset_index()
df_arrival_delay = df_arrival_delay.rename(columns={'ARRIVAL_DELAY': 'AVERGAE_ARRIVAL_DELAY(min)'})

#Merging df to Airlines_Ranking
Airlines_Ranking=pd.merge(Airlines_Ranking,df_arrival_delay,how='inner')
Airlines_Ranking.head()

,AIRLINE_NAME,OPERATED_SCHEDULED_%,AVERAGE_FLIGHT_SPEED(miles/hour),AVERGAE_ARRIVAL_DELAY(min)
0,Alaska Airlines Inc.,99.612221,435.510696,-0.976563
1,American Airlines Inc.,98.495972,427.679334,3.451372
2,American Eagle Airlines Inc.,94.900418,360.899443,6.457873
3,Atlantic Southeast Airlines,97.337131,364.683738,6.585379
4,Delta Air Lines Inc.,99.563411,419.433004,0.186754


In [53]:
#Flight volume of each airline i.e. number of flights

#Dataframe with total number of flights for each airlines
df_flight_volume =pd.DataFrame(Flights.groupby('AIRLINE_NAME').count()['FLIGHT_NUMBER'])
df_flight_volume=df_flight_volume.reset_index()
df_flight_volume = df_flight_volume.rename(columns={'FLIGHT_NUMBER': 'FLIGHT_COUNT'})

#Total number of flights of all airlines
total_flights_all_airlines=df_flight_volume['FLIGHT_COUNT'].sum()

df_flight_volume['FLIGHT_COUNT']=df_flight_volume['FLIGHT_COUNT']/total_flights_all_airlines*100
df_flight_volume = df_flight_volume.rename(columns={'FLIGHT_COUNT': 'FLIGHT_VOLUME(%)'})

#Merging df to Airlines_Ranking
Airlines_Ranking=pd.merge(Airlines_Ranking,df_flight_volume,how='inner')
Airlines_Ranking.head()

,AIRLINE_NAME,OPERATED_SCHEDULED_%,AVERAGE_FLIGHT_SPEED(miles/hour),AVERGAE_ARRIVAL_DELAY(min),FLIGHT_VOLUME(%)
0,Alaska Airlines Inc.,99.612221,435.510696,-0.976563,2.964748
1,American Airlines Inc.,98.495972,427.679334,3.451372,12.475926
2,American Eagle Airlines Inc.,94.900418,360.899443,6.457873,5.063207
3,Atlantic Southeast Airlines,97.337131,364.683738,6.585379,9.829339
4,Delta Air Lines Inc.,99.563411,419.433004,0.186754,15.051884


In [54]:
#Average Taxi-In and Taxi-Out of each airline in minutes

Flights_with_Taxi_In_Out=Flights.dropna(subset=['TAXI_IN','TAXI_OUT']) #Dropping rows with null values

df_taxi_in_out =pd.DataFrame(Flights_with_Taxi_In_Out.groupby('AIRLINE_NAME').mean()['TAXI_IN'])
df_taxi_in_out['TAXI_OUT']=Flights_with_Taxi_In_Out.groupby('AIRLINE_NAME')['TAXI_OUT'].mean()
df_taxi_in_out=df_taxi_in_out.reset_index()
df_taxi_in_out = df_taxi_in_out.rename(columns={'TAXI_IN': 'AVERGAE_TAXI_IN(min)','TAXI_OUT':'AVERGAE_TAXI_OUT(min)'})

#Merging df to Airlines_Ranking
Airlines_Ranking=pd.merge(Airlines_Ranking,df_taxi_in_out,how='inner')
Airlines_Ranking.head()

,AIRLINE_NAME,OPERATED_SCHEDULED_%,AVERAGE_FLIGHT_SPEED(miles/hour),AVERGAE_ARRIVAL_DELAY(min),FLIGHT_VOLUME(%),AVERGAE_TAXI_IN(min),AVERGAE_TAXI_OUT(min)
0,Alaska Airlines Inc.,99.612221,435.510696,-0.976563,2.964748,6.401667,15.096324
1,American Airlines Inc.,98.495972,427.679334,3.451372,12.475926,8.876172,17.767420
2,American Eagle Airlines Inc.,94.900418,360.899443,6.457873,5.063207,9.084096,16.544693
3,Atlantic Southeast Airlines,97.337131,364.683738,6.585379,9.829339,7.635511,16.734140
4,Delta Air Lines Inc.,99.563411,419.433004,0.186754,15.051884,7.209058,17.607960


In [55]:
# Creating a score for each airline

#Score is directly proportional to Operated_Scheduled%,Average_Flight_Speed, Flight_Volume
#Score is inversely proportional to Average_Arrival_Delay, Average_Taxi_In, Average_Taxi_Out

#Higher the score the best the airlines is

Airlines_Ranking['Airline_Score']=(((Airlines_Ranking['OPERATED_SCHEDULED_%']/100)*Airlines_Ranking['AVERAGE_FLIGHT_SPEED(miles/hour)']*(Airlines_Ranking['FLIGHT_VOLUME(%)']/100))/((Airlines_Ranking['AVERGAE_ARRIVAL_DELAY(min)'].abs()*Airlines_Ranking['AVERGAE_TAXI_IN(min)']*Airlines_Ranking['AVERGAE_TAXI_OUT(min)'])))
Airlines_Ranking.reset_index()
Airlines_Ranking.head()

,AIRLINE_NAME,OPERATED_SCHEDULED_%,AVERAGE_FLIGHT_SPEED(miles/hour),AVERGAE_ARRIVAL_DELAY(min),FLIGHT_VOLUME(%),AVERGAE_TAXI_IN(min),AVERGAE_TAXI_OUT(min),Airline_Score
0,Alaska Airlines Inc.,99.612221,435.510696,-0.976563,2.964748,6.401667,15.096324,0.136281
1,American Airlines Inc.,98.495972,427.679334,3.451372,12.475926,8.876172,17.767420,0.096553
2,American Eagle Airlines Inc.,94.900418,360.899443,6.457873,5.063207,9.084096,16.544693,0.017867
3,Atlantic Southeast Airlines,97.337131,364.683738,6.585379,9.829339,7.635511,16.734140,0.041466
4,Delta Air Lines Inc.,99.563411,419.433004,0.186754,15.051884,7.209058,17.607960,2.651531


In [ ]:
cwd = os.getcwd() #To access the path of current working directory
path=cwd+'/Analysis-1'

In [89]:
#Plot of OPERATED_SCHEDULED_% v/s Airline

Airlines_Ranking=Airlines_Ranking.apply(pd.to_numeric, errors='ignore')

ax1=sns.pointplot(x="AIRLINE_NAME", y="OPERATED_SCHEDULED_%", data=Airlines_Ranking, figsize=(15,10))
plt.title('Percentage of Operated to Scheduled Flights of each Airlines')
ax1.set(xlabel='Airline Name', ylabel='Operated to Scheduled %')
plt.xticks(rotation=20)
plt.show()
plt.savefig(path+'/'+'Percentage_of_Operated_to_Scheduled_Flights_of_each_Airlines.jpeg', format="JPEG")

In [90]:
#Plot of AVERAGE_FLIGHT_SPEED v/s Airline

ax2=Flights_with_Distance_Airtime[['AIRLINE_NAME','FLIGHT_SPEED']].boxplot(column = 'FLIGHT_SPEED',by='AIRLINE_NAME',figsize=(15,10),rot=20)
ax2.get_figure().suptitle('') # To remove auto-generated box-plot title
plt.title('Average Flight Speed of each Airlines')
plt.show()
plt.savefig(path+'/'+'Average_Flight_Speed_of_each_Airlines.jpeg', format="JPEG")

In [96]:
#Plot of AVERAGE_ARRIVAL_DELAY v/s Airline

plt.figure(figsize=(15,10))
ax=sns.barplot(x="AIRLINE_NAME", y="AVERGAE_ARRIVAL_DELAY(min)", data=Airlines_Ranking,color='blue')
plt.title('Average Arrival Delay(minutes) of each Airlines')
ax.set(xlabel='Airline Name', ylabel='Average Arrival Delay(minutes)')
plt.xticks(rotation=20)
plt.show()
plt.savefig(path+'/'+'Average_Arrival_Delay(minutes)_of_each_Airlines.jpeg', format="JPEG")

In [94]:
#Plot of FLIGHT_VOLUME(%) v/s Airline

#ax=sns.(x="AIRLINE_NAME", y="AVERGAE_ARRIVAL_DELAY(min)", data=Airlines_Ranking,color='gray')
ax=Airlines_Ranking['FLIGHT_VOLUME(%)'].plot.pie(figsize=(15,10),rot=45,labels=Airlines_Ranking['AIRLINE_NAME'])
ax.set_ylabel('')
plt.title('Flight Volume(%) of each Airlines')
plt.show()
plt.savefig(path+'/'+'Flight_Volume(%)_of_each_Airlines.jpeg', format="JPEG")

In [95]:
#Plot of AVERGAE_TAXI_IN(min), AVERGAE_TAXI_OUT(min) v/s Airline
plt.figure(figsize=(15,10))
pos = list(range(len(Airlines_Ranking['AIRLINE_NAME'])))
width = 0.25

# Plotting the bars
fig, ax = plt.subplots(figsize=(14,6))
# Create a bar with Taxi_In
plt.bar(pos,Airlines_Ranking['AVERGAE_TAXI_IN(min)'],width,alpha=0.5,color='blue',label=Airlines_Ranking['AIRLINE_NAME'])

# Create a bar with Taxi_Out
plt.bar([p + width for p in pos],Airlines_Ranking['AVERGAE_TAXI_OUT(min)'],width,alpha=0.5,color='red',label=Airlines_Ranking['AIRLINE_NAME'])

# Set the y axis label
ax.set_ylabel('Time in minutes')

# Set the chart's title
ax.set_title('Average Taxi-In and Taxi-Out(in minutes) of each Airlines')

# Set the position of the x ticks
ax.set_xticks([p + 1.5 * width for p in pos])

# Set the labels for the x ticks
ax.set_xticklabels(Airlines_Ranking['AIRLINE_NAME'])

# Setting the x-axis and y-axis limits
plt.xlim(min(pos)-width, max(pos)+width*4)
plt.ylim([0, max( Airlines_Ranking['AVERGAE_TAXI_IN(min)'] +  Airlines_Ranking['AVERGAE_TAXI_IN(min)'])] )

# Adding the legend and showing the plot
plt.legend(['Taxi-In', 'Taxi-Out'], loc='upper right')
plt.grid()
plt.xticks(rotation=12)
plt.show()
plt.savefig(path+'/'+'Average_Taxi-In_and_Taxi-Out(in_minutes)_of_each_Airlines.jpeg', format="JPEG")

In [65]:
#Ranking Airlines based on their score
#Airline with highes score is best
Airlines_Ranking_Sorted=Airlines_Ranking.sort_values(['Airline_Score'], ascending=[False])
Airlines_Ranking_Sorted=Airlines_Ranking_Sorted.reset_index(drop=True)
Airlines_Ranking_Sorted['Ranking']=Airlines_Ranking_Sorted.index+1
Airlines_Ranking_Sorted.head()

,AIRLINE_NAME,OPERATED_SCHEDULED_%,AVERAGE_FLIGHT_SPEED(miles/hour),AVERGAE_ARRIVAL_DELAY(min),FLIGHT_VOLUME(%),AVERGAE_TAXI_IN(min),AVERGAE_TAXI_OUT(min),Airline_Score,Ranking
0,Delta Air Lines Inc.,99.563411,419.433004,0.186754,15.051884,7.209058,17.607960,2.651531,1
1,Southwest Airlines Co.,98.728618,415.197481,4.374964,21.684789,6.167662,11.943144,0.275829,2
2,Alaska Airlines Inc.,99.612221,435.510696,-0.976563,2.964748,6.401667,15.096324,0.136281,3
3,American Airlines Inc.,98.495972,427.679334,3.451372,12.475926,8.876172,17.767420,0.096553,4
4,Skywest Airlines Inc.,98.307139,368.598815,5.845652,10.110758,6.933260,18.142272,0.049827,5


In [68]:
#Writing the output to csv

if not os.path.exists(path+'/'+'Analysis_1_Ouput_Ranking_of_Airlines.csv'):
    Airlines_Ranking_Sorted.to_csv(path+'/'+'Analysis_1_Ouput_Ranking_of_Airlines.csv', sep=',',index=False)
else:
    print('File already exists.')

In [88]:
#Plot of Airline_Score v/s Airline
plt.figure(figsize=(15,10))
ax=sns.barplot(x="AIRLINE_NAME", y='Airline_Score', data=Airlines_Ranking_Sorted,color='blue')

plt.title('Airline Score of each Airlines')
ax.set(xlabel='Airline Name', ylabel='Airline_Score')
plt.xticks(rotation=20)
plt.show()
plt.savefig(path+'/'+'Airline_Score_of_each_Airlines.jpeg', format="JPEG")

Conclusion:

By above observations, Delta Air Lines Inc. is the best and Frontier Airlines Inc. is the worst airlines in US as of 2015.